In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
URL = 'https://www.indeed.com/jobs?q=data+scientist&l=New+York&start=10'
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')
#print(soup)

In [ ]:
max_results_per_city = 23500
city_set = ['New York','Chicago','San Francisco', 'Austin','San Antonio' 'Seattle', 'Los Angeles', 'Philadelphia', 'Atlanta',
            'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington DC']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'desired_experience', 'salary']
sample_df = pd.DataFrame(columns = columns)
print(sample_df)

In [ ]:
#scraping code:
for city in city_set:
    for start in range(0, 5000, 10):
        page = requests.get('https://www.indeed.com/jobs?q=data+scientist&l=' + str(city) + '&start=' + str(start))
        time.sleep(2)  # wait 2 second between page grabs
        soup = BeautifulSoup(page.text, 'lxml')
        for div in soup.find_all(name='div', attrs={'class':'row'}): 
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating list
            job_post = [] 
            #append city name
            job_post.append(city)
            #grabbing job title
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                job_post.append(a['title']) 
                #adding company
                company = div.find_all(name='span', attrs={'class':'company'}) 
                if len(company) > 0: 
                    for b in company:
                        job_post.append(b.text.strip()) 
                else: 
                    sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                    for span in sec_try:
                        job_post.append(span.text) 
            
            #adding location
            c = div.findAll('span', attrs={'class': 'location'}) 
            for span in c: 
                job_post.append(span.text) 

            #adding summary
            d = div.findAll('span', attrs={'class': 'summary'}) 
            for span in d:
                job_post.append(span.text.strip()) 
            
            #adding Desired Experience
            
            e = div.findAll('span', attrs={'class': 'experienceList'})
            if len(e) >0:
                for span in e:
                    job_post.append(span.text.strip())
            else:
                job_post.append('NA')
                   
                
            #adding salary
            try:
                job_post.append(div.find('nobr').text) 
            except:
                try:
                    div2 = div.find(name='div', attrs={'class':'sjcl'}) 
                    div3 = div2.find('div') 
                    job_post.append(div3.text.strip())
                except:
                    job_post.append('NA') 
            #putting job post to dataframe at index num
            sample_df.loc[num] = job_post
            #print(sample_df)
            

#saving sample_df as a csv file 
sample_df.to_csv('indeed_test1.csv')
